# 递归

### 递归是什么
- 递归是一种解决问题的方法
- 将问题分解为规模更小的相同问题
- 持续分解，直到问题规模小道可以用非常简单直接的方式解决
- 算法方面显著的特征就是在算法流程中调用自身

### 递归的法则
1. 递归算法必须有一个基本结束条件（最小规模问题的直接解决）
2. 递归算法必须能改变状态项基本结束条件演进（减小问题规模）
3. 递归算法必须调用自身（解决减小了规模的相同问题）

### 递归的应用1：任意进制转换

In [2]:
def toStr(n, base):
    converString = "0123456789ABCDEF"
    if n < base:
        return converString[n] #最小规模问题解法
    else:
        return toStr(n//base, base) + converString[n%base] #调用自身缩小问题并加上该位余数

In [3]:
print(toStr(1453, 16))

5AD


### 递归的实现：
当一个函数被调用的时候，系统会把调用时的现场数据压入到系统调用栈
- 每次调用，压入栈的现场数据称为栈帧
- 当函数返回时，要从调用栈的栈顶取得返回地址，恢复现场，弹出栈帧，按地址返回

在调试递归算法程序的时候经常会碰到错误：RecursionError

这是说明递归的层数太多，系统调用栈容量有限，可以通过sys模块获取和调整最大递归深度

In [5]:
import sys
sys.getrecursionlimit() #获取递归深度

3000

In [6]:
sys.setrecursionlimit(3500) #调整递归深度
sys.getrecursionlimit()

3500

In [13]:
#使用turtle模块来可视化递归
import turtle

def drawSpiral(t, lineLen): #画螺旋线
    if lineLen > 0:
        t.forward(lineLen)
        t.right(90)
        drawSpiral(t, lineLen - 5)

In [14]:
t = turtle.Turtle()
drawSpiral(t, 100)
turtle.done()

Terminator: 

### 递归的应用2：分形学
绘制分形树，该部分内容代码放在tree.py中


谢尔宾斯基三角形，该部分内容代码放在sierpinski.py中

### 递归的应用3：汉诺塔
解题思路，对于N层塔片的汉诺塔问题 ：
- 首先将上层N-1个盘片，从开始柱，经由目标柱，移动到中间柱（递归）
- 将第N个盘片，从开始组，移动到目标柱
- 最后将中间柱的N-1个盘片，经开始柱，移动到目标柱（递归）
- 结束条件是只剩一个盘片的移动问题

In [17]:
def moveTower(height, fromPole, withPole, toPole):
    if height >= 1:
        moveTower(height - 1, fromPole, toPole, withPole) #把N-1个盘片经由目标柱移动到中间柱的递归
        moveDisk(height, fromPole, toPole)
        moveTower(height - 1, withPole, fromPole, toPole) #把N-1个盘片经由开始柱移动到目标柱的递归

def moveDisk(disk, fromPole, toPole):
    print(f'移动碟片{disk} from {fromPole} to {toPole}')

In [18]:
moveTower(4, '#1', '#2', '#3')

移动碟片1 from #1 to #2
移动碟片2 from #1 to #3
移动碟片1 from #2 to #3
移动碟片3 from #1 to #2
移动碟片1 from #3 to #1
移动碟片2 from #3 to #2
移动碟片1 from #1 to #2
移动碟片4 from #1 to #3
移动碟片1 from #2 to #3
移动碟片2 from #2 to #1
移动碟片1 from #3 to #1
移动碟片3 from #2 to #3
移动碟片1 from #1 to #2
移动碟片2 from #1 to #3
移动碟片1 from #2 to #3


### 递归的应用4：迷宫
采用矩阵方式来实现迷宫数据结构

In [19]:
class Maze:
    def __init__(self, mazeFileName):
        rowsInMaze = 0
        columnsInMaze = 0
        self.mazelist = []
        mazeFile = open(mazeFileName, 'r')
        for line in mazeFile:
            rowList = []
            col = 0
            for ch in line[: -1]:
                rowList.append(ch)
                if ch == 'S':
                    self.startRow = rowsInMaze
                    self.startCol = col
                col = col + 1
            rowsInMaze = rowsInMaze + 1
            self.mazelist.append(rowList)
            columnsInMaze = len(rowList)

海龟探索迷宫的思路：
- 将海龟从原位置**向北**移一步，以**新位置**递归调用探索迷宫寻找出口
- 如果上面步骤找不到出口，那么将海龟从原位置**向南**移动一步，以**新位置**递归调用探索迷宫
- 如果向南找不到出口，将海龟从原位置**向西**一步，以**新位置**递归调用探索迷宫
- 如果向西找不到出口，将海龟从原位置**向东**移动一步，以新位置递归调用探索迷宫

以上思路的问题是，如果海龟北面两步是墙，那么海龟在遇到墙后就会向南回退一步，回到原点，继而陷入了无限递归的死循环
- 因此需要有机制记录海龟走过的路径
- 沿途撒“面包屑”，如果前进方向发现“面包屑”，就不能再踩上去，必须换下一个方向尝试
- 对于递归调用来说，就是某方向的方格上发现“面包屑”，就立即从递归调用**返回上一级**

因此将该递归的结束条件，总结如下：
- 碰到“墙壁”方格，递归调用结束，返回失败
- 碰到“面包屑”方格，表示此方格已经访问过，递归调用结束，返回失败
- 在四个方向上探索都失败，递归调用结束，返回失败
- 碰到“出口”方格，递归调用结束，返回成功